<a href="https://colab.research.google.com/github/davidmerwin/Google_Quantum_Deep_Learning_Training/blob/main/docs/fqe/guide/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The OpenFermion Developers

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to FQE

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/openfermion/fqe/guide/introduction"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/OpenFermion/blob/master/docs/fqe/guide/introduction.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/OpenFermion/blob/master/docs/fqe/guide/introduction.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/OpenFermion/docs/fqe/guide/introduction.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

OpenFermion-FQE is an emulator for quantum computing specialized for simulations of Fermionic many-body problems, where FQE stands for 'Fermionic quantum emulator.' By focusing on Fermionic physics, OpenFermion-FQE realizes far more efficient emulation of quantum computing than generic quantum emulators such as Cirq, both in computation and memory costs; the speed-up and improved memory footprint originate from the use of the spin and number symmetries as well as highly optimized special algorithms.

The examples of the problems that can be simulated by OpenFermion-FQE include those in molecular electronic structure, condensed matter physics, and nuclear physics.

The initial version of OpenFermion-FQE has been developed in collaboration between [QSimulate](https://qsimulate.com)  and [Google Quantum AI](https://quantumai.google/). The source code is found in the GitHub repository (https://github.com/quantumlib/OpenFermion-FQE).

This tutorial will describe the data structures and conventions of the library.

In [2]:
try:
    import fqe
except ImportError:
    !pip install fqe --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.1/579.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00


In [3]:
import fqe
import numpy as np

## The FQE Wavefunction

The `Wavefunction` is an interface to the objects that hold the actual wavefunction data.  As mentioned, the wavefunction is partitioned into sectors with fixed particle and $Sz$ quantum number.  This partitioning information is the necessary information for initializing a `Wavefunction` object.  

As an example, we consider initializing a wavefunction with four spatial orbitals, four electrons, and different $Sz$ expectation values.  The `Wavefunction` object takes a list of these sectors `[[n_electrons, sz, n_orbits]]`.

In [4]:
wfn = fqe.Wavefunction([[4, 4, 4], [4, 2, 4], [4, 0, 4], [4, -2, 4], [4, -4, 4]])

This command initializes a wavefunction with the following block structure:

![Image of wf sectors](https://github.com/quantumlib/OpenFermion/blob/master/docs/fqe/guide/wf_init_sectors.png?raw=1)

Each sector corresponds to a set of bit strings

$$
\vert I \rangle = \vert I_{\alpha}I_{\beta}\rangle
$$

that encode a fixed particle number and fixed $Sz$ expectation. The coefficients associated with the bitstrings in these sectors are formed into matrices.  This helps with efficient vectorized computations. The row-index of the array corresponds to the $\alpha$ spin-orbital number occupation index and the column-index corresponds to the $\beta$-strings.  The `Wavefunction` object provides tools to access sectors or perform basic mathematical operations on this *vector*.  

## Methods to initialize wavefunctions

FQE wavefunctions can be initialized by calling the constructor directly.

In [5]:
wfn_fqe = fqe.Wavefunction([[2, -2, 4]], broken=None)

When wavefunctions are first created, they are initialized to empty values. We can see this by printing out the wavefunction.

In [6]:
wfn_fqe.print_wfn()

Sector N = 2 : S_z = -2


To set the values of a wavefunction, we can use the `set_wfn` method with a provided `strategy`.

In [7]:
wfn_fqe.set_wfn(strategy="hartree-fock")
wfn_fqe.print_wfn()

Sector N = 2 : S_z = -2
a'0000'b'0011' (1+0j)


Users can access the wavefunction through the `get_sector` method.  This returns the entire matrix of data representing the specified sector of the wavefunction.  For example, we can grab the sector corresponding to $n = 2$ and $sz = -2$ by doing the following.

In [8]:
interesting_states = wfn_fqe.get_coeff((2, -2))
print(interesting_states)

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


Other than the `Wavefunction` constructor, several utility methods are available to initialize wavefunctions. The function `fqe.get_wavefunction` builds a wavefunction with definite particle number and spin.

In [9]:
wfn_fqe = fqe.get_wavefunction(4, -2, 10)

The function `fqe.get_wavefunction_multiple` constructs multiple wavefunctions with different particle number, spin, and orbital number.

In [10]:
wfn_fqe1, wfn_fqe2 = fqe.get_wavefunction_multiple([[4, 0, 10], [5, -5, 10]])

There are also functions like `fqe.get_number_conserving_wavefunction` and  `fqe.get_spin_conserving_wavefunction` to get number or spin conserving wavefunctions, respectively.

In [11]:
# Get a spin conserving wavefunction.
spin_conserving_wfn = fqe.get_spin_conserving_wavefunction(2, 4)

# Get a number conserving wavefunction.
number_conserving_wfn = fqe.get_number_conserving_wavefunction(2, 4)

### Conversions between FQE and Cirq wavefunction representations

Wavefunctions on $n$ qubits in Cirq are represented by Numpy arrays with $2^n$ amplitudes.

In [12]:
nqubits = 4

cirq_wfn = np.random.rand(2**nqubits) + 1.0j * np.random.rand(2**nqubits)
cirq_wfn /= np.linalg.norm(cirq_wfn)

print("Cirq wavefunction:")
print(*cirq_wfn, sep="\n")

Cirq wavefunction:
(0.035381404179289755+0.22188532016666784j)
(0.33240482279388833+0.06781801995663009j)
(0.2595460115952728+0.15095454665766772j)
(0.13217386545963178+0.20848300991786356j)
(0.14154615786759717+0.10169659783275856j)
(0.2195662003538022+0.08832073659097137j)
(0.15889861600425298+0.24967525091553824j)
(0.20709201148454054+0.021461616470677784j)
(0.04284722467757695+0.02190525290884709j)
(0.051693287967068534+0.11945703075157675j)
(0.3236578703286288+0.1955406208036639j)
(0.21565665540362133+0.30571656243283324j)
(0.01626671687508401+0.11939708071175004j)
(0.1891665909675747+0.05956970379761574j)
(0.22547006021917035+0.15460695851859504j)
(0.13470193514833229+0.13060769434307334j)


To convert from this representation to the FQE representation, the function `fqe.from_cirq` can be used.

In [13]:
fqe_wfn = fqe.from_cirq(cirq_wfn, thresh=0.0001)
fqe_wfn.print_wfn()

Sector N = 0 : S_z = 0
a'00'b'00' (0.035381404179289755+0.22188532016666784j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.14154615786759717+0.10169659783275856j)
a'00'b'10' (0.33240482279388833+0.06781801995663009j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.04284722467757695+0.02190525290884709j)
a'10'b'00' (0.2595460115952728+0.15095454665766772j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.2195662003538022+0.08832073659097137j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.01626671687508401+0.11939708071175004j)
a'01'b'10' (0.051693287967068534+0.11945703075157675j)
a'10'b'01' (-0.15889861600425298-0.24967525091553824j)
a'10'b'10' (0.13217386545963178+0.20848300991786356j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.3236578703286288+0.1955406208036639j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.1891665909675747+0.05956970379761574j)
a'10'b'11' (-0.20709201148454054-0.021461616470677784j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.22547006021917035-0.15460695851859504j)
a'11'b'10' (0.21565665540362133+0.30571656243283324j)
Se

> *Note*: The `thresh` argument is the value below which amplitudes are considered zero.

We can convert back to the Cirq representation using `fqe._to_cirq`.

In [14]:
cirq_wfn_from_fqe = fqe.to_cirq(fqe_wfn)

print("Cirq wavefunction from FQE:")
print(*cirq_wfn_from_fqe, sep="\n")

Cirq wavefunction from FQE:
(0.035381404179289755+0.22188532016666784j)
(0.33240482279388833+0.06781801995663009j)
(0.2595460115952728+0.15095454665766772j)
(0.13217386545963178+0.20848300991786356j)
(0.14154615786759717+0.10169659783275856j)
(0.2195662003538022+0.08832073659097137j)
(0.15889861600425298+0.24967525091553824j)
(0.20709201148454054+0.021461616470677784j)
(0.04284722467757695+0.02190525290884709j)
(0.051693287967068534+0.11945703075157675j)
(0.3236578703286288+0.1955406208036639j)
(0.21565665540362133+0.30571656243283324j)
(0.01626671687508401+0.11939708071175004j)
(0.1891665909675747+0.05956970379761574j)
(0.22547006021917035+0.15460695851859504j)
(0.13470193514833229+0.13060769434307334j)


In [15]:
assert np.allclose(cirq_wfn_from_fqe, cirq_wfn)

An important thing to note in these conversions is the ordering of the $\alpha$ and $\beta$ strings in the converted wavefunctions.  The FQE uses the OpenFermion convention of interleaved $\alpha$ and $\beta$ orbitals.  Thus when converting to Cirq we first convert each bitstring into an OpenFermion operator and then call normal ordering.

### Printing and saving wavefunctions

Printing is currently available as alpha beta strings followed by the coefficient as well as orbital occupation representation.

In [16]:
print('String formatting')
fqe_wfn.print_wfn(fmt='str')

String formatting
Sector N = 0 : S_z = 0
a'00'b'00' (0.035381404179289755+0.22188532016666784j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.14154615786759717+0.10169659783275856j)
a'00'b'10' (0.33240482279388833+0.06781801995663009j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.04284722467757695+0.02190525290884709j)
a'10'b'00' (0.2595460115952728+0.15095454665766772j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.2195662003538022+0.08832073659097137j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.01626671687508401+0.11939708071175004j)
a'01'b'10' (0.051693287967068534+0.11945703075157675j)
a'10'b'01' (-0.15889861600425298-0.24967525091553824j)
a'10'b'10' (0.13217386545963178+0.20848300991786356j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.3236578703286288+0.1955406208036639j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.1891665909675747+0.05956970379761574j)
a'10'b'11' (-0.20709201148454054-0.021461616470677784j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.22547006021917035-0.15460695851859504j)
a'11'b'10' (0.21565665540362133+0.3057

In [17]:
print('Occupation formatting')
fqe_wfn.print_wfn(fmt='occ')

Occupation formatting
Sector N = 0 : S_z = 0
.. (0.035381404179289755+0.22188532016666784j)
Sector N = 1 : S_z = -1
.b (0.14154615786759717+0.10169659783275856j)
b. (0.33240482279388833+0.06781801995663009j)
Sector N = 1 : S_z = 1
.a (0.04284722467757695+0.02190525290884709j)
a. (0.2595460115952728+0.15095454665766772j)
Sector N = 2 : S_z = -2
bb (0.2195662003538022+0.08832073659097137j)
Sector N = 2 : S_z = 0
.2 (0.01626671687508401+0.11939708071175004j)
ba (0.051693287967068534+0.11945703075157675j)
ab (-0.15889861600425298-0.24967525091553824j)
2. (0.13217386545963178+0.20848300991786356j)
Sector N = 2 : S_z = 2
aa (0.3236578703286288+0.1955406208036639j)
Sector N = 3 : S_z = -1
b2 (0.1891665909675747+0.05956970379761574j)
2b (-0.20709201148454054-0.021461616470677784j)
Sector N = 3 : S_z = 1
a2 (-0.22547006021917035-0.15460695851859504j)
2a (0.21565665540362133+0.30571656243283324j)
Sector N = 4 : S_z = 0
22 (-0.13470193514833229-0.13060769434307334j)


Wavefunctions can also be saved to disk using the `save` method which takes a filename and optional path.

# Action on Wavefunctions: Fermionic algebra operations and their unitaries on the state

FermionOperators can be directly passed in to create a new wavefunction based on application of the operators.  The FermionOperators are parsed according to the interleaved $\alpha$ $\beta$ indexing of the spin-orbitals.  This means that odd index FermionOperators correspond to $\beta$-spin orbitals and even are $\alpha$-spin orbitals.  

Sharp Edge:

The user must be careful to not break the symmetry of the wavefunction.  If a request to apply an operator to a state takes the wavefunction outside of the specified symmetry sector the FQE will not execute the command.  Effectively, the FQE requires the user to have more knowledge of what type of operations their `Wavefunction` object can support.

In [18]:
from openfermion import FermionOperator, hermitian_conjugated

ops = FermionOperator('2^ 0', 1.2)

new_wfn = fqe_wfn.apply(ops + hermitian_conjugated(ops))

## Unitary operations

Any simulator backend must be able to perform unitary evolution on a state.  The FQE accomplishes this by implementing code for evolving a state via the action of a unitary generated by fermionic generators.  Given a fermion operator $g$, the unitary

$$
e^{-i (g + g^{\dagger})}
$$

can be applied to the state.  It can be shown that this evolution operator can be rewritten as

$$
e^{-i(g + g^{\dagger})\epsilon } = \cos\left(\epsilon\right) \mathbb{I}_{s}(gg^{\dagger}) +  \cos\left(\epsilon\right) \mathbb{I}_{s}(g^{\dagger}g) - i\sin\left(\epsilon\right) \left(g + g^{\dagger}\right) \left[\mathbb{I}_{s}(gg^{\dagger}) + \mathbb{I}_{s}(g^{\dagger}g)\right] + \mathbb{I}_{!s}
$$

The $\mathbb{I}_{!s}$ is for setting the coefficients of the unitary that are not in the subspace $\mathcal{H}_{s} \subset \mathcal{H}$ where $gg^{\dagger}$ is 0.   

The user can specify a fermionic monomial in OpenFermion and use the `time_evolve` method of the `Wavefunction` object to call the evolution. All the rules for preserving symmetries must be maintained as before.

In [19]:
i, j, theta = 0, 1, np.pi / 3
op = (FermionOperator(((2 * i, 1), (2 * j, 0)), coefficient=-1j * theta) +
      FermionOperator(((2 * j, 1), (2 * i, 0)), coefficient=1j * theta))

new_wfn = fqe_wfn.time_evolve(1.0, op)

new_wfn.print_wfn()

Sector N = 0 : S_z = 0
a'00'b'00' (0.035381404179289755+0.22188532016666784j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.14154615786759717+0.10169659783275856j)
a'00'b'10' (0.33240482279388833+0.06781801995663009j)
Sector N = 1 : S_z = 1
a'01'b'00' (-0.20334982715364822-0.11977784576788j)
a'10'b'00' (0.16687979085007756+0.09444777882421843j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.2195662003538022+0.08832073659097137j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.14574359652341365+0.27592365034498506j)
a'01'b'10' (-0.08861928122089341-0.12082306747052451j)
a'10'b'01' (-0.06536191795213474-0.021436720423692615j)
a'10'b'10' (0.11085463331444169+0.20769432825045617j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.3236578703286288+0.1955406208036639j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.27393023835021824+0.04837115696869336j)
a'10'b'11' (0.06027706758294932+0.04085806854931068j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.22547006021917035-0.15460695851859504j)
a'11'b'10' (0.21565665540362133+0.30571656243283324j)
Sec

In other tutorials we will do a deeper dive into supported time-evolution operations.  To serve a full functioning time-evolution platform the FQE also implements arbitrary time-evolution of full Hamiltonian operators consisting of sums of non-commuting terms.  The Taylor and Chebyshev expansion methods are used to do the exact time evolution.  